In [1]:
import os
import pandas as pd
import numpy as np  
import importlib as imp
import shutil
import random
import swg as swg
imp.reload(swg)

<module 'swg' from 'c:\\Users\\micha\\Documents\\swgPythonJunk\\swg.py'>

	objectName = "@mob/creature_names:brackaset",
	socialGroup = "brackaset",
	faction = "",
	mobType = MOB_HERBIVORE,
	level = 26,
	x chanceHit = 0.36,
	x damageMin = 240,
	x damageMax = 250,
	x baseXp = 2637,
	x baseHAM = 8100,
	x baseHAMmax = 9900,
	armor = 0,
	x resists = {30,30,120,30,30,-1,-1,-1,-1},
	meatType = "meat_wild",
	meatAmount = 100,
	hideType = "hide_leathery",
	hideAmount = 91,
	boneType = "bone_mammal",
	boneAmount = 81,
	milkType = "milk_wild",
	milk = 50,
	tamingChance = 0.25,
	ferocity = 0,
	pvpBitmask = ATTACKABLE,
	creatureBitmask = PACK + HERD,
	optionsBitmask = AIENABLED,
	diet = HERBIVORE,


### To Hit
This works to create the to-hit value. For some bosses it might need special edits.

In [2]:
def getToHit_orig(original):
    # define the ranges
    min_start, max_start = 1, 600
    min_end, max_end = 0.02, 36

    # check if the original number is in the provided range
    if not (min_start <= original <= max_start):
        raise ValueError(f"Number must be between {min_start} and {max_start}, inclusive.")

    # calculate the range sizes
    start_range = max_start - min_start
    end_range = max_end - min_end

    # map the original number into the target range
    scaled = ((original - min_start) / float(start_range)) * end_range + min_end

    # add some randomness to the mapped number
    final_number = random.uniform(max(scaled-1, min_end), scaled+1)

    # return the result, rounded to two decimal places
    return round(final_number, 2)




def getToHit(n):
    BREAKPOINTS = [
        {"max": 20, "mean": 0.275, "std_dev": 0.0875, "min_val": 0.1, "max_val": 0.45},
        {"max": 100, "mean": 1.4, "std_dev": 0.55, "min_val": 0.3, "max_val": 2.5},
        {"max": 300, "mean": 16, "std_dev": 8, "min_val": 2.51, "max_val": 32},
        {"max": 600, "mean": 32.5, "std_dev": 16.25, "min_val": 32.01, "max_val": 65},
    ]
    if n < 1 or n > 600:
        raise ValueError("Input should be between 1 and 600")
    
    for bp in BREAKPOINTS:
        if n < bp["max"]:
            value = np.random.normal(bp["mean"], bp["std_dev"])
            clipped_value = np.clip(value, bp["min_val"], bp["max_val"])
            return round(clipped_value, 2)
    
    return None

In [3]:

def getResists(CL):

    list_length = 8
    result_list = [-1]*list_length

    level_multiplied = CL * 10

    min_sum = level_multiplied * 0.2
    max_sum = level_multiplied * 0.3

    forbidden_ranges = [(95, 100)]
    column_numbers = 2

    if 30 <= CL <= 70:
        column_numbers = 4
    elif CL > 70:
        column_numbers = 5

    columns_to_fill = set([0] + random.sample(range(1, list_length), column_numbers-1))

    current_sum = 0

    while len(columns_to_fill) > 0:
        selected_column = columns_to_fill.pop()
        while True:
            random_num = round(random.uniform(0, min(95, CL)), 1)
            if any(lower <= random_num <= upper for lower, upper in forbidden_ranges):
                continue
            if current_sum + random_num > max_sum:
                continue
            else:
                break
        if random_num > 0:
            random_num = max(5, random_num)

        result_list[selected_column] = random_num
        current_sum += random_num

        if current_sum >= min_sum:
            break

    random.shuffle(result_list)
    if result_list[0] <= 0:
        minimum_value = max(1, CL/2)  # Lower limit is maximum of 1 and CL/2
        new_value = random.uniform(minimum_value, min(95, CL))

        while any(lower <= new_value <= upper for lower, upper in forbidden_ranges):  # Avoid forbidden ranges
            new_value = round(new_value - 0.1, 1)
            if new_value <= 0:  
                new_value = -1  
                break

        result_list[0] = round(new_value, 1)
    return result_list

def getResists2(level):  
    list_length = 8  
    level_multiplied = level * 10  
  
    min_sum = level_multiplied * 0.2  
    max_sum = level_multiplied * 0.3  
  
    forbidden_ranges = [(95, 100)]  
    result_list = []  
    counter = 0  
  
    # Calculate target number of positions with a value of 5 or greater  
    target_positions = round(level / 10)  + 2
    target_positions += random.randint(-2, 2)
    target_positions = max(min(target_positions, 7), 3)
    target_positions = 7
  
    while counter < 1000:    # Loop will stop after 1,000 iterations  
        counter += 1  
  
        icounter = 0  
        while True:  
            icounter += 1  
            if icounter > 10000:  
                break  
            random_num = round(random.uniform(0, min(95, level)), 1)  
            if any(lower <= random_num <= upper for lower, upper in forbidden_ranges):  
                continue  
            break  
    
        current_sum = sum(result_list)  
        if current_sum + random_num > max_sum:  
            continue  
        if random_num > 0:  # Ensure minimum value is 5 if the number is greater than 0  
            random_num = max(5, random_num)  
            
        result_list.append(random_num)  
  
        current_sum = sum(result_list)  
        list_len = len(result_list)  
        if list_len == list_length:  
            if current_sum < min_sum or sum(val >= 5 for val in result_list) != target_positions:  # Check if the number of positions with a value of 5 or greater matches the target  
                result_list = []  
            else:  
                break  
        elif current_sum >= min_sum:  
            break  
    while len(result_list) < list_length:  
        result_list.append(-1)  
  
    random.shuffle(result_list)  
    if result_list[0] <= 0:
        result_list[0] = 35
  
    return result_list

def getResists_orig(level):  
    list_length = 8  
    level_multiplied = level * 10  
  
    min_sum = level_multiplied * 0.2  
    max_sum = level_multiplied * 0.3  
  
    forbidden_ranges = [(95, 100), (195, 200)]  
    result_list = []  
    counter = 0  
  
    # Calculate target number of positions with a value of 5 or greater  
    target_positions = round(level / 10)  + 2
    target_positions += random.randint(-2, 2)
    if target_positions > 7:
        target_positions = 7
    elif target_positions < 3:
        target_positions = 3   
  
    while counter < 1000:    # Loop will stop after 1,000 iterations  
        counter += 1  
  
        icounter = 0  
        while True:  
            icounter += 1  
            if icounter > 10000:  
                break  
            random_num = round(random.uniform(0, min(95, level)), 1)  
            if any(lower <= random_num <= upper for lower, upper in forbidden_ranges):  
                continue  
            if random_num > 50:  
                if random.random() < 0.25:  
                    random_num = random_num# + 100  
                break  
  
        current_sum = sum(result_list)  
        if current_sum + random_num > max_sum:  
            continue  
        if random_num == 0 or (level < 60 and random.random() < 0.05):  # Added condition to increase -1 positions for levels below 60  
            random_num = -1  
        elif random_num > 0:  # Ensure minimum value is 5 if the number is greater than 0  
            random_num = max(5, random_num)  
            
        result_list.append(random_num)  
  
        current_sum = sum(result_list)  
        list_len = len(result_list)  
        if list_len == list_length:  
            if current_sum < min_sum or sum(val >= 5 for val in result_list) != target_positions:  # Check if the number of positions with a value of 5 or greater matches the target  
                result_list = []  
            else:  
                break  
        elif current_sum >= min_sum:  
            break  

    while len(result_list) < list_length:  
        result_list.append(-1)  
  
    random.shuffle(result_list)  
  
    return result_list  

import random  

def getSimResists(old_list, old_level, new_level):  
    if old_level == -1 or new_level == -1:  
        return [-1]*len(old_list)  

    forbidden_ranges = [(95, 100), (195, 200)]  
    proportion = new_level / old_level  

    new_level = min(max(new_level, 1), 600) 

    new_list = []  
    
    increase = new_level > old_level

    for value in old_list:
        if value <= 0 and increase and random.random() < (new_level - old_level)/100:  # added chance for a value change based on level increase
            value = random.uniform(1, min(95, new_level))

        new_value = round(value * proportion, 1)  

        if new_value > 100:  
            new_value = random.uniform(85, 95)
        
        while any(lower <= new_value <= upper for lower, upper in forbidden_ranges) or new_value > 200.0:
            new_value = round(new_value - 0.1, 1)
            if new_value <= 0.0:  
                new_value = -1  
                break

        multiplier = random.uniform(0.92, 1.08)  
        new_value *= multiplier   
        new_value = round(new_value, 1)
        
        while any(lower <= new_value <= upper for lower, upper in forbidden_ranges) or new_value > 200.0:
            new_value = round(new_value - 0.1, 1)
            if new_value <= 0:  
                new_value = -1  
                break  
        
        if new_value > 95:  
            new_value = random.uniform(85, 95)
            new_value = round(new_value, 1) 

        if new_value <= 0:  
            new_value = value

        new_list.append(new_value)  

    return new_list

def getSimResists_orig(old_list, old_level, new_level):  
  
    if old_level == -1 or new_level == -1:  
        return [-1]*len(old_list)  
  
    forbidden_ranges = [(95, 100), (195, 200)]  
    proportion = new_level / old_level  
  
    if new_level < 1:  
        new_level = 1  
  
    if new_level > 600:  
        new_level = 600  
  
    new_list = []  
  
    for value in old_list:  
          
        new_value = round(value * proportion, 1)  
  
        if new_value > 100:  
            new_value = random.uniform(85, 95)  
          
        while any(lower <= new_value <= upper for lower, upper in forbidden_ranges) or new_value > 200.0:  
            new_value = round(new_value - 0.1, 1)  
            if new_value <= 0.0:  
                new_value = -1  
                break  
              
        multiplier = random.uniform(0.92, 1.08)  
        new_value *= multiplier   
        new_value = round(new_value, 1)  
          
        while any(lower <= new_value <= upper for lower, upper in forbidden_ranges) or new_value > 200.0:  
            new_value = round(new_value - 0.1, 1)  
            if new_value <= 0:  
                new_value = -1  
                break  
  
        if new_value > 95:  
            new_value = random.uniform(85, 95) 
            new_value = round(new_value, 1) 
  
        if new_value <= 0:  
            new_value = value#-1  
        new_list.append(new_value)  
  
    return new_list  


In [4]:
  
def add_bonus_resist(resist_list, level):  
    chance = level / 10
    new_resist_list = []
    bonus_list = []
  
    for resist in resist_list:  
        if resist >= 200:
            resist -= 200
        if resist > 95 and resist <= 99:
            resist = resist - 10
        if resist == -1 or resist == 0:
            if random.random() * 100 < 40:  
                new_resist_list.append(resist)
            else:
                new_resist_list.append(0)
            bonus_list.append(False)
            continue
        rax = random.random() * 100
        
        if rax < chance:  
            new_resist_list.append(resist + 100)
            bonus_list.append(True)
        else:
            new_resist_list.append(resist)
            bonus_list.append(False)
      
    return new_resist_list, bonus_list 

def applyBonus(bonus_list, rlist):
    new_list = []
    for i in range(len(bonus_list)):
        if bonus_list[i]:
            if rlist[i] < 100:
                new_list.append(rlist[i] + 100)
            else:
                new_list.append(rlist[i])
        else:
            # approximately 50% chance to add 100 to rlist[i] if it's over 85
            if rlist[i] > 85 and random.random() < 0.5:
                new_list.append(rlist[i] + 100)
            else:
                new_list.append(rlist[i])
    return new_list


In [5]:

def getHamRange(creature_level):  
    first_number = creature_level * 2.35 * 100  
    first_number += first_number * random.uniform(-0.2, 0.2)  
      
    second_number = (first_number * 0.27) 
    second_number += second_number * random.uniform(0, 0.27)  
      
    return round(first_number,0), round(first_number + second_number,0) 
  
  
def getDamage(level):    
    base_number = level * 8    
    lower_bound = base_number * 0.85    
    upper_bound = base_number * 1.75    
      
    damageMin = random.uniform(lower_bound, upper_bound - (level * 2.2))  
    damageMax = damageMin + (level * 2.2)  
      
    return round(damageMin, 0), round(damageMax, 0)  

def getDamageBuild(damageMin, damageMax, old_level, new_level):  
    ratio = new_level / old_level  
    random_factor_min = 1 - random.uniform(0, 0.15)  
    random_factor_max = 1 - random.uniform(0, 0.15)  
  
    new_damageMin = damageMin * ratio * random_factor_min  
    new_damageMax = damageMax * ratio * random_factor_max  
  
    return round(new_damageMin, 0), round(new_damageMax, 0)  

def getBaseXp(level):  
    guide_levels = [1, 31, 65, 345, 600]  
    guide_xp_min = [5, 2500, 6000, 22000, 55000]  
    guide_xp_max = [15, 3500, 6500, 32000, 65000]  
  
    xp_min = np.interp(level, guide_levels, guide_xp_min)  
    xp_max = np.interp(level, guide_levels, guide_xp_max)  
  
    xp_value = random.uniform(xp_min, xp_max)  
    return round(xp_value, 0)  
  
level = 1  # You can change this to any valid level (1 to 600)  
xp_value = getBaseXp(level)  
print(xp_value)  

def nameFix(name):
    vowels = ['a', 'e', 'i', 'o', 'u']  # list of vowels
    name = name.lower().replace('_', ' ')  # replace underscore with space
    if name[0] in vowels:  # check if the first letter is a vowel
        return 'an ' + name
    else:
        return 'a ' + name

def nameFile(input_string):
    # replacing spaces with _
    modified_string = input_string.lower().replace(' ', '_')
    
    # removing anything that is not a letter or a number
    modified_string = ''.join(ch for ch in modified_string if ch.isalnum() or ch == '_')
    
    return modified_string

def harvests(old_number, old_level, new_level):
    level_difference = new_level - old_level
    random_number = random.randint(4, 9)
    result = old_number + (level_difference * random_number)
    return result


6.0


In [6]:
def creoTemplate(row):
    temp = []
    temp.append(row['name'] + ' = Creature:new {')
    temp.append('\tobjectName = "@mob/creature_names:' + row['name'] + '",')
    temp.append('\tsocialGroup = "' + row['social'] + '",')
    temp.append('\tfaction = "",')
    temp.append('\tmobType = ' + row['mobtype'] + ',')
    temp.append('\tlevel = ' + str(round(row['CL'])) + ',')
    temp.append('\tchanceHit = ' + str(row['tohit']) + ',')
    temp.append('\tdamageMin = ' + str(round(row['dmgMin'])) + ',')
    temp.append('\tdamageMax = ' + str(round(row['dmgMax'])) + ',')
    temp.append('\tspecialDamageMult = 1,')
    temp.append('\tbaseXp = ' + str(round(row['xp'])) + ',')
    temp.append('\tbaseHAM = ' + str(round(row['hamMin'])) + ',')
    temp.append('\tbaseHAMmax = ' + str(round(row['hamMax'])) + ',')
    temp.append('\tarmor = ' + str(round(row['armor'])) + ',')
    resists = [str(item) for item in row['resists']]
    temp.append('\tresists = {' + ','.join(resists) + ',-1},')
    if pd.isna(row['meatamt']):
        temp.append('\tmeatType = "",')
        temp.append('\tmeatAmount = 0,')
    else:
        temp.append('\tmeatType = "' + row['meat'] + '",')
        temp.append('\tmeatAmount = ' + str(round(row['meatamt'])) + ',')

    if pd.isna(row['hideamt']):
        temp.append('\thideType = "",')
        temp.append('\thideAmount = 0,')
    else:
        temp.append('\thideType = "' + row['hide'] + '",')
        temp.append('\thideAmount = ' + str(round(row['hideamt'])) + ',')

    if pd.isna(row['boneamt']):
        temp.append('\tboneType = "",')
        temp.append('\tboneAmount = 0,')
    else:
        temp.append('\tboneType = "' + row['bone'] + '",')
        temp.append('\tboneAmount = ' + str(round(row['boneamt'])) + ',')


    temp.append('\tmilk = 0,')
    temp.append('\ttamingChance = 0,')
    temp.append('\tferocity = 0,')
    temp.append('\tpvpBitmask = ATTACKABLE,')
    temp.append('\tcreatureBitmask = NONE,')
    temp.append('\toptionsBitmask =  AIENABLED,')
    temp.append('\tdiet = ' + row['diet'] + ',')
    tmps = row['templates'].split(',')
    temp.append('\ttemplates = {' + ','.join(f'"{item}"' for item in tmps) + '},')
    temp.append('\tcontrolDeviceTemplate = "",')
    temp.append('\tscale = ' + str(row['scale']) + ',')
    temp.append('\tlootGroups = {},')
    temp.append('\tconversationTemplate = "",')
    temp.append('\tprimaryWeapon = "unarmed",')
    temp.append('\tsecondaryWeapon = "none",')
    temp.append('\tprimaryAttacks = { },')
    temp.append('\tsecondaryAttacks = { },')
    temp.append("}")
    temp.append('CreatureTemplates:addCreatureTemplate(' + row['name'] + ', "' + row['name'] + '")')
    return temp

# Read Input Excel

In [7]:
dfmain = pd.read_excel("input/creatureInput.xlsx")
dfmain.head()
creolist = []
namelist = []
stringList = []

The goal here is to add all the stats to these creatures and then start a new excel file

The trick is to process each creo stub at a time. So first we get a list of the unique mob social groups which is the core mob type.

In [8]:
def creoRunner(mob):
    df = dfmain[dfmain.socialGroup == mob]

    primaryCL = 0
    lvllist = []
    clist = []
    resists = []
    tohits = []
    hams = []
    hamsl = []
    hamsh = []
    dmgd = []
    dmgl = []
    dmgh = []
    xpx = []
    names = []
    socials = []
    mobtype = []
    diets = []
    armors = []
    scales = []
    meattypes = []
    meatamounts = []
    hidetypes = []
    hideamounts = []
    bonetypes = []
    boneamounts = []
    templates = []
    presists = []
    n = 0
    m = 0
    for i in range(len(df)):
        if not pd.isna(df.iloc[i,2]):
            m = i
            if not pd.isna(df.iloc[i,0]):
                if n == 0:
                    primaryCL = df.iloc[i,0]
                    presists = getResists(primaryCL)
                    bresists,bonus_list = add_bonus_resist(presists,df.iloc[i,0])
                    resists.append(bresists)

                    dmg = getDamage(primaryCL)
                    dmgl.append(dmg[0])
                    dmgh.append(dmg[1])
                    dmgd.append(dmg)
                else:
                    sresists = getSimResists(resists[0],primaryCL,df.iloc[i,0])
                    tresists = applyBonus(bonus_list, sresists)
                    resists.append(tresists)
                    dmg = getDamageBuild(dmgd[0][0],dmgd[0][1],primaryCL,df.iloc[i,0])
                    dmgd.append(dmg)
                    dmgl.append(dmg[0])
                    dmgh.append(dmg[1])
                    dmgd.append(dmg)
                n+=1
                if df.iloc[i,4] == 'h':
                    mobtype.append('MOB_HERBIVORE')
                    diets.append('HERBIVORE')
                else:
                    mobtype.append('MOB_CARNIVORE')
                    diets.append('CARNIVORE')
                clist.append(nameFix(df.iloc[i,1]))
                names.append(nameFile(df.iloc[i,1]))
                stringList.append(nameFile(df.iloc[i,1]) + '\t' + nameFix(df.iloc[i,1]))
                armors.append(df.iloc[i,5])
                scales.append(df.iloc[i,6])
                socials.append(df.iloc[i,2])
                lvllist.append(df.iloc[i,0])
                tohits.append(getToHit(df.iloc[i,0]))
                ham = getHamRange(df.iloc[i,0])
                hams.append(ham)
                hamsl.append(ham[0])
                hamsh.append(ham[1])
                xpx.append(getBaseXp(df.iloc[i,0]))
                meattypes.append(df.iloc[0,7])
                hm = harvests(df.iloc[0,8], primaryCL, df.iloc[i,0])
                hh = harvests(df.iloc[0,10], primaryCL, df.iloc[i,0])
                hb = harvests(df.iloc[0,12], primaryCL, df.iloc[i,0])
                if hm < 0:
                    hm = 10
                if hh < 0:
                    hh = 10
                if hb < 0:
                    hb = 10
                
                meatamounts.append(hm)
                hidetypes.append(df.iloc[0,9])
                hideamounts.append(hh)
                bonetypes.append(df.iloc[0,11])
                boneamounts.append(hb)
                templates.append(df.iloc[i,13])
                    
    dfo = pd.DataFrame(list(zip(lvllist,names,socials,clist,resists,tohits,hamsl,hamsh,dmgl,dmgh,xpx,mobtype,diets,meattypes,meatamounts,hidetypes,hideamounts,bonetypes,boneamounts,templates,scales,armors)),columns=['CL','name','social','creo','resists','tohit','hamMin','hamMax','dmgMin', 'dmgMax','xp','mobtype','diet','meat','meatamt','hide','hideamt','bone','boneamt','templates','scale','armor'])
    
    for i in range(len(dfo)):
        a = creoTemplate(dfo.iloc[i])
        f = open("output/mobiles/" + dfo.iloc[i,1] + ".lua", "w")
        f.write('\n'.join(a))
        f.close()
        creolist.append('includeFile("../custom_scripts/mobile/kashyyyk/' + dfo.iloc[i,1] + '.lua")')
        namelist.append(str(dfo.iloc[i,0]) + ' ' + dfo.iloc[i,1])

In [9]:
socialgroups = []
for i in range(len(dfmain)):
    if not pd.isna(dfmain.iloc[i,2]):
        if dfmain.iloc[i,2] not in socialgroups:
            socialgroups.append(dfmain.iloc[i,2])

print(socialgroups)

['kashyyyk_bantha', 'mouf', 'uller', 'walluga', 'webweaver', 'bolotaur', 'Jyykle', 'ro-roo', 'sathog', 'pug', 'ysalamiri', 'katarn', 'kkorrwrot', 'kklyyytt', 'minstyngar']


In [10]:
for s in socialgroups:
    creoRunner(s)

f = open("output/mobiles/serverobjects.lua", "w")
f.write('\n'.join(creolist))
f.close()

f = open("input/creoList", "w")
f.write('\n'.join(namelist))
f.close()

In [ ]:
for s in stringList:
    print(s)

# Wild Lairs


In [232]:
def create_creature_list(creature_list, qty=30):
    results = []
    
    # Initialize creature lists outside loop
    creature_list.sort(key=lambda x: x[0])
    low_level_creatures = [c for c in creature_list if c[0] < 60]
    high_level_creatures = [c for c in creature_list if c[0] >= 60]
    used_creatures = []

    for _ in range(qty):
        # Reinitialize creature pools if all creatures have been used
        if len(used_creatures) == len(creature_list):
            low_level_creatures = [c for c in creature_list if c[0] < 60]
            high_level_creatures = [c for c in creature_list if c[0] >= 60]
            used_creatures = []

        current_list = []
        num_of_creatures = random.randint(2, 4)

        # Randomly select low level creatures
        for _ in range(num_of_creatures - 1):
            if low_level_creatures:
                creature = random.choice(low_level_creatures)
                low_level_creatures.remove(creature)
                used_creatures.append(creature)
                current_list.append(creature)
            elif creature_list:
                creature = random.choice(creature_list)
                creature_list.remove(creature)
                used_creatures.append(creature)
                current_list.append(creature)

        # If there are high level creatures and list is not empty, add one high level creature to the current list
        if high_level_creatures and current_list:
            creature = max(high_level_creatures)
            high_level_creatures.remove(creature)
            used_creatures.append(creature)
            current_list.append(creature)

        current_list[:-1] = [(f'{creature[1]}', random.randint(2, 3)) for creature in current_list[:-1]]
        
        if current_list:
            current_list[-1] = (f'{current_list[-1][1]}', 1)

        results.append(', '.join([f'{creature[0]}, {creature[1]}' for creature in current_list]))

    return results

the following uses the method above and creates a list of randomized lair spawns you can then use to copy/paste into the associated long list to generate:
input/kashLairlist.txt

In [ ]:
creatures = """10	kklyyytt
8	kklyyytt_whimp
12	noodleleg_kklyyytt
15	kklyyytt_spriteling
22	kklyyytt_weanling
27	tripping_kklyyyt
31	kklyyytt_bumblefoot
42	clumsy_kklyyytt
54	squawking_kklyyytt
126	minstyngar_devastator"""

creature_list = [item.split('\t') for item in creatures.split('\n')]
nlist = []
for c in creature_list:
    nlist.append([int(c[0]),c[1]])

lairlist = create_creature_list(nlist,40)
for l in lairlist:
    print(l)

In [260]:
def lairTemplateWild(label,creos):
    temp = []
    temp.append(label + ' = Lair:new {')
    clist = creos.split(',')
    result = []

    # process elements in pair of two
    for i in range(0, len(clist), 2):
        # append each pair to the result list with the correct formatting
        result.append('{{"{}",{}}}'.format(clist[i].strip(), clist[i + 1].strip()))

    # join all elements in result list into a string
    output = ",".join(result)
    temp.append('\tmobiles = {' + output.strip() + '},')
    temp.append('\tspawnLimit = 16,')
    temp.append('\tbuildingsVeryEasy = {},')
    temp.append('\tbuildingsEasy = {},')
    temp.append('\tbuildingsMedium = {},')
    temp.append('\tbuildingsHard = {},')
    temp.append('\tbuildingsVeryHard = {},')
    temp.append('\tbuildingType = "none",')
    temp.append('}')
    temp.append('addLairTemplate("' + label + '", ' + label + ')')

    return temp

In [279]:
with open('input/kashWildLairlist.txt', 'r') as f:
    lines = f.readlines()
print(str(len(lines)))

heading = ""
lairs = []
lnames = []
i=1
for l in lines:
    if len(l.strip()) > 0:
        lc = l.split(',')
        if len(lc) > 1:
            la = heading.replace('XXX',str(i)).strip() 
            lnames.append(la) 
            t = lairTemplateWild(la ,l)
            f = open("output/lairs/creature_dynamic/" + la + ".lua", "w")
            f.write('\n'.join(t))
            f.close()
            lairs.append('includeFile("../custom_scripts/mobile/lair/creature_dynamic/' + la + '.lua")')
        else:
            la = l.replace('XXX',str(i)).strip()
            heading = l
            i = 0
        i+=1
    
    


In [277]:
f = open("output/lairs/creature_dynamic/serverobjects.lua", "w")
f.write('\n'.join(lairs))
f.close()
f = open("input/lairsList.txt", "w")
f.write('\n'.join(lnames))
f.close()

# Spawns
This is the groups of lairs

In [22]:
def getSpawn(lair):
    temp = []
    temp.append('\t\t{')
    temp.append('\t\tlairTemplateName = "' + lair.strip() + '",')
    temp.append('\t\tspawnLimit = -1,')
    temp.append('\t\tminDifficulty = 18,')
    temp.append('\t\tmaxDifficulty = 500,')
    temp.append('\t\tnumberToSpawn = 15,')
    temp.append('\t\tweighting = 8,')
    temp.append('\t\tsize = 15,')
    temp.append('\t\t},')
    return '\n'.join(temp)

In [29]:
def makeSpawns(spawn,filters):
    temp = []
    temp.append(spawn.strip() + "= {")
    temp.append('\tlairSpawns = {')
    filepath = 'input/lairsComb.txt'
    with open(filepath, 'r') as file:
        lines = file.readlines()
    for line in lines:
        for filter_word in filters:
            if filter_word in line:
                temp.append(getSpawn(line))
    temp.append("\t}")
    temp.append("}")
    temp.append('addSpawnGroup("' + spawn.strip() + '", ' + spawn.strip() + ');')
    return '\n'.join(temp)


In [30]:

name = "inf_kash_hard"
spawn = makeSpawns(name, ["hard"])
f = open("output/spawn/" + name + ".lua", "w")
f.write(spawn)
f.close()

# Lairs - Non-Wild

In [17]:
def lairTemplate(data,creos):
    temp = []
    headings = data.split('|')
    label = headings[0].strip()
    count = headings[1].strip()
    style = headings[2].strip()
    ct = creos.split(';')
    clist = str(ct[0]).split(',')
    blist = []
    result = []
    bresult = []
    if len(ct) > 1:
        blist = str(ct[1]).split(',')
        if len(blist) > 0:
            for i in range(0, len(blist), 2):
                # append each pair to the result list with the correct formatting
                bresult.append('{{"{}",{}}}'.format(blist[i].strip(), blist[i + 1].strip()))


    temp.append(label + ' = Lair:new {')
    # process elements in pair of two
    for i in range(0, len(clist), 2):
        # append each pair to the result list with the correct formatting
        result.append('{{"{}",{}}}'.format(clist[i].strip(), clist[i + 1].strip()))

    # join all elements in result list into a string
    output = ",".join(result)
    temp.append('\tmobiles = {' + output.strip() + '},')

    if len(bresult) > 0:
        boutput = ",".join(bresult)
        temp.append('\tbossMobiles = {' + boutput.strip() + '},')

    temp.append('\tspawnLimit = ' + str(count) + ',')
    temp.append('\tbuildingsVeryEasy = {"' + style + '"},')
    temp.append('\tbuildingsEasy = {"' + style + '"},')
    temp.append('\tbuildingsMedium = {"' + style + '"},')
    temp.append('\tbuildingsHard = {"' + style + '"},')
    temp.append('\tbuildingsVeryHard = {"' + style + '"},')

    temp.append('}')
    temp.append('addLairTemplate("' + label + '", ' + label + ')')

    return temp

In [20]:
with open('input/kashLairlist.txt', 'r') as f:
    lines = f.readlines()
print(str(len(lines)))

heading = ""
lairs = []
lnames = []
i=1
data = ""
for l in lines:
    if len(l.strip()) > 0:

        lc = l.split(',')
        if len(lc) > 1:
            dl = data.split('|')
            la = dl[0].replace('XXX',str(i)).strip() 
            lnames.append(la) 
            t = lairTemplate(data.replace('XXX',str(i)) ,l)
            f = open("output/lairs/creature_lair/" + la + ".lua", "w")
            f.write('\n'.join(t))
            f.close()
            lairs.append('includeFile("../custom_scripts/mobile/lair/creature_lair/' + la + '.lua")')
        else:
            data = l            
            i = 0
        i+=1

253


In [21]:
f = open("output/lairs/creature_lair/serverobjects.lua", "w")
f.write('\n'.join(lairs))
f.close()
f = open("input/lairsList.txt", "w")
f.write('\n'.join(lnames))
f.close()